# Sylhet Floods 2022
## calc_fracArea_matchedToFusion.ipynb
This script calculates fractional inundated area stats for the maps outputs which have already been resampled to 500 m resolution and aligned consistent with the fusion model outputs.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import rioxarray as riox
import xarray
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Set the module path for helpers scripts
module_path = os.path.abspath(os.path.join('C:/Users/alexa/Documents/GitHub/Sylhet2022Floods/scr/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import module from analyse_modifiedDevries containing helpful functions to use
import importlib
import helpers.prepare_flood_raster as prep_raster
importlib.reload(prep_raster)
import helpers.analyse_modifiedDevries as analyse_DV
importlib.reload(analyse_DV)

<module 'Helpers.analyse_modifiedDevries' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\analyse_modifiedDevries.py'>

## PART 1: Get the matched rasters

In [6]:
matchedPath = rootPath/'Raster/SylhetMatchedToFusion'

In [7]:
FusionPath=matchedPath/'Fusion'
GFDPath=matchedPath/'GFD'
DevriesPath=matchedPath/'Devries'
IMPACTPath=matchedPath/'IMPACT'
GFMPath=matchedPath/'GFM'
UNOSATPath=matchedPath/'UNOSAT'

## PART 2: Load spatial files for defining the ROI

In [8]:
# Import the ROI geometry from shapefile
adminShapePath = rootPath/'Shapefiles/AdminHDX'
districts = gpd.read_file(adminShapePath/'bgd_admbnda_adm2_bbs_20201113.shp')
sylhet_dist = districts[districts['ADM2_EN']=='Sylhet'].reset_index()

# Get the ROI as shapely Polygon geom
geom_dist = sylhet_dist.loc[0, 'geometry']

## PART 3: Get the mean, median and 5/95%ile FIA values

In [10]:
# For GFD we will runs two versions using different band outputs of the GFD
GFDBands=[0,4]
GFDBandsNames=['flooded','max_img']

In [21]:
# Loop through all map sources and through all map dates for each
MapDataPaths = [FusionPath, GFDPath,DevriesPath,IMPACTPath,GFMPath,UNOSATPath]

# Create an empty dataframe for saving results
FIAStatsDF = pd.DataFrame(data=[], index=['MapSource','ImageDate','ImageBand','MeanFIA','MedianFIA','StdevFIA','05PctFIA','95PctFIA']).T

for MapDataPath in MapDataPaths:
    
    # Map source
    MapSource=MapDataPath.stem
    
    # Get all the files
    MapDataFiles = [file for file in list(MapDataPath.iterdir()) if file.suffix=='.tif' and '2022' in str(file)]
    
    # Loop through files
    for MapDataFile in MapDataFiles:
        
        # Map date
        ImageDate=MapDataFile.stem
        
        # If source is GFD, do two versions: one with band0 'flooded' and one with band4 'max_img'
        if MapDataPath==GFDPath:
            
            for i, GFDBand in enumerate(GFDBands):
                
                # Get the FIA values for the file and get stats
                FIAStats = analyse_DV.getFIAStats(analyse_DV.getFIAValsROI(MapDataFile, geom_dist, GFDBand))
                
                # Append results to dataframe
                FIAStatsDF = pd.concat(objs=[FIAStatsDF, 
                                             pd.DataFrame([MapSource, ImageDate, GFDBandsNames[i]] + FIAStats, index=FIAStatsDF.columns).T], 
                                                          axis=0, ignore_index=True)
                
        else:    
        
            # Get the FIA values for the file and get stats
            FIAStats = analyse_DV.getFIAStats(analyse_DV.getFIAValsROI(MapDataFile, geom_dist))
            
            # Append results to dataframe
            FIAStatsDF = pd.concat(objs=[FIAStatsDF, 
                                         pd.DataFrame([MapSource, ImageDate, 'NA'] + FIAStats, index=FIAStatsDF.columns).T], 
                                                      axis=0, ignore_index=True)

In [22]:
FIAStatsDF

MapSource ImageDate ImageBand   MeanFIA MedianFIA  StdevFIA  05PctFIA  \
0      Fusion  20220504        NA  0.090027  0.072737  0.065253  0.021923   
1      Fusion  20220512        NA  0.167151  0.140528  0.135239  0.012278   
2      Fusion  20220520        NA   0.41722  0.390283    0.3324  0.018233   
3      Fusion  20220528        NA  0.457646  0.422678  0.349128  0.018326   
4      Fusion  20220605        NA   0.46204  0.469115  0.288645  0.034838   
..        ...       ...       ...       ...       ...       ...       ...   
101       GFM  20220815        NA  0.115046       0.0  0.219965       0.0   
102       GFM  20220827        NA  0.120676       0.0  0.222065       0.0   
103       GFM  20220829        NA  0.116793       0.0  0.222094       0.0   
104    UNOSAT  20220525        NA  0.370871  0.193113  0.400744       0.0   
105    UNOSAT  20220619        NA  0.389225  0.246673  0.398547       0.0   

     95PctFIA  
0    0.211306  
1    0.423461  
2    0.896861  
3     0.96349  
4    0.883284  
..        ...  
101   0.65153  
102   0.65269  
103   0.65269  
104       1.0  
105       1.0  

[106 rows x 8 columns]

In [23]:
# Export the results as csv
outputPath = rootPath/'Table/SylhetFracFloodedArea'
outputPath.mkdir(exist_ok=True)
pd.DataFrame.to_csv(FIAStatsDF, outputPath/'FIA_matchedToFusion_sylhetDist.csv', index=False)